In [89]:
from __future__ import absolute_import, division, print_function

import sys
import os
import numpy
import math

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import leastsq
from scipy.optimize import curve_fit
import scipy
from matplotlib.backends.backend_pdf import PdfPages
import glob
    
sys.path.insert(0,os.path.abspath(os.path.join('..', r'C:\\jupyter\\GitHub\\adc-testing\\adc-testing\\SignalProcessing\\')))

import SingleToneSignal

def fit_sin(tt, yy):
    '''Fit sin to the input time sequence, and return fitting parameters "amp", "omega", "phase", "offset", "freq", "period" and "fitfunc"'''
    tt = numpy.array(tt)
    yy = numpy.array(yy)
    ff = numpy.fft.fftfreq(len(tt), (tt[1]-tt[0]))   # assume uniform spacing
    Fyy = abs(numpy.fft.fft(yy))
    guess_freq = abs(ff[numpy.argmax(Fyy[1:])+1])   # excluding the zero frequency "peak", which is related to offset
    guess_amp = numpy.std(yy) * 2.**0.5
    guess_offset = numpy.mean(yy)
    guess = numpy.array([guess_amp, 2.*numpy.pi*guess_freq, 0., guess_offset])

    def sinfunc(t, A, w, p, c):  return A * numpy.sin(w*t + p) + c
    popt, pcov = scipy.optimize.curve_fit(sinfunc, tt, yy, p0=guess)
    A, w, p, c = popt
    f = w/(2.*numpy.pi)
    fitfunc = lambda t: A * numpy.sin(w*t + p) + c
    return {"amp": A, "omega": w, "phase": p, "offset": c, "freq": f, "period": 1./f, "fitfunc": fitfunc, "maxcov": numpy.max(pcov), "rawres": (guess,popt,pcov)}

def procData(afe_out, freq=6.54E10, show=True):
    #afe_out = np.loadtxt(foutName)
    N, amp, omega, phase, offset, noise = len(afe_out), 1., 2., .5, 4., 3
    tt = numpy.linspace(0, 1024*16/freq, N)
    tt2 = numpy.linspace(0, 1024*16/freq, 10*N)
    afe_out = afe_out

    res = fit_sin(tt, afe_out)
    if show: print( "Frequency=%(freq)s, Amplitude=%(amp)s, Angular freq.=%(omega)s, phase=%(phase)s, offset=%(offset)s, Max. Cov.=%(maxcov)s" % res )
    err=res["fitfunc"](tt)-afe_out
    max_out=max(afe_out)
    min_out=min(afe_out)
    max_err = max(err)
    min_err = min(err)
    abs_err = max_err - min_err
    if show: print( "AbsErr=%s"%abs_err, "MinErr%s"%min_err, "MaxErr%s"%max_err,"Max=%s"%max_out, "Min=%s"%min_out)


    plt.plot(tt, afe_out, "ok", label="ADC Output")
    plt.plot(tt, err, ".b", label="Err")
    plt.plot(tt2, res["fitfunc"](tt2), "r-", label="Fit curve", linewidth=1)
    plt.legend(loc="best")
    if show: plt.show()
    return max_out, min_out, err, max_err, min_err, abs_err

def fftData(afe_out, freq=6.54E10, N=None, show=False):
    #afe_out = np.loadtxt(foutName)
    fs = freq
    signal = afe_out
    #signal = signal * 1.0 / 128                     # Scale the signal in [-1, 1]
    
    if N == None:
        N = len(afe_out)

    win = np.hamming(N)                                                       
    x = signal[0:N] #* win                             # Take a slice and multiply by a window
    
    sp = np.fft.rfft(x)                               # Calculate real FFT
    #For an input of size N the rfft function returns the part of the FFT output corresponding 
    #to frequences at or below N/2. Therefore the output of rfft is of size N/2+1 if N is even 
    #(all frequences from 0 to N/2), or (N+1)/2 if N is odd (all frequences from 0 to (N-1)/2). 
    #Observe that the function floor(n/2+1) returns the correct output size for both even and odd input sizes.
    
    mag = np.abs(sp) 
    ref = np.sum(win) / 2                             # Reference : window sum and factor of 2
                                                      # because we are using half of FFT spectrum
    
    s_dbfs = 20 * np.log10(mag / ref)                 # Convert to dBFS

    freq = np.arange((N / 2) + 1) / (float(N) / fs)   # Frequency axis
    if show:
        plt.plot(freq, s_dbfs)
        plt.grid(True)
        plt.xlabel('Frequency [Hz]')
        plt.ylabel('Amplitude [dB]')
        plt.show()
    plt.clf()
    return freq, s_dbfs

def processSignal(nbits = 8, rate = 6.54E10, data = None):
	output = SingleToneSignal.SingleToneSignal(nbits =nbits, rate = rate, data = data)
	output.precalculateAll()
	for each in output.items(): print(each[0], each[1]%each[2])
	for each in output.windowed['No window'].items(): print(each[0], each[1]%each[2])
	return output

def removeOffsetGain(dout, calFile=None):
	# Remove Offset and Gain
	dout_corr = np.copy(dout)
	N=len(dout)
	if calFile==None:
		dc = np.zeros(64, dtype='f')
		gm = np.zeros(64, dtype='f')
		for j in range(64):
			dc[j] = np.mean(dout[j::64])
			gm[j] = np.linalg.norm(dout_corr[j::64])/math.sqrt(N/64)
	else:
		aaa=np.genfromtxt(calFile,delimiter=',',dtype='f')
		dc=aaa[0][1:]
		gm=aaa[1][1:]
	for j in range(64):
		dout_corr[j::64] = dout[j::64]-dc[j]
		dout_corr[j::64] = dout_corr[j::64]*(gm[0]/gm[j])
	return dout_corr

def procAll(afe_out, fname='.\\',samplingRate=10.3125*5*34/33*1.0e9, show = False, cal=False, calFile=None, pdf=None, png=False):
    if cal:
        afe_out_corr=removeOffsetGain(afe_out,calFile)
    else:
        afe_out_corr=afe_out
    nbits=8
    samplingRate=samplingRate
    output1 = processSignal(nbits = nbits, rate = samplingRate, data = afe_out)
    chartData = fftData(afe_out=afe_out, freq=samplingRate, N=output1.nsamples)
    output2 = processSignal(nbits = nbits, rate = samplingRate, data = afe_out_corr)
    chartDataCorr = fftData(afe_out=afe_out_corr, freq=samplingRate, N=output2.nsamples)
    for i in range(len(output1.items())):
    	print(output1.items()[i][0] +','+ output1.items()[i][1]%output1.items()[i][2] +','+ output1.items()[i][1]%output2.items()[i][2])

    fig = plt.figure(1)
    fig.set_size_inches(18.5,12.5)
    axs1 = plt.subplot2grid((3, 2), (0, 0))
    axs2 = plt.subplot2grid((3, 2), (0, 1))
    axs3 = plt.subplot2grid((3, 2), (1, 0))
    axs4 = plt.subplot2grid((3, 2), (1, 1))
    axs5 = plt.subplot2grid((3, 2), (2, 0), colspan=2)

    T=1/samplingRate

    xf=np.arange(0.0,samplingRate/2-1,samplingRate/2/len(output1.windowed['No window'].ldft))
    axs1.plot(xf,output1.windowed['No window'].ldft)
    axs1.grid(True)
    axs1.set_xlabel('Frequency [Hz]')
    axs1.set_ylabel('Amplitude [dB]')
    axs1.set_ylim([-80,140])

    xf=np.arange(0.0,samplingRate/2-1,samplingRate/2/len(output2.windowed['No window'].ldft))
    axs2.plot(xf,output2.windowed['No window'].ldft)
    axs2.grid(True)
    axs2.set_xlabel('Frequency [Hz]')
    axs2.set_ylabel('Amplitude [dB]')
    axs2.set_ylim([-80,140])

    N=int(output1.items()[2][2])
    inFreq=float(output1.items()[3][2])
    nSamples1T = int(1/inFreq * samplingRate)
    timePlot = nSamples1T * 3
    tt = numpy.linspace(0, 1024*16/samplingRate, N)
    tt2 = numpy.linspace(0, 1024*16/samplingRate, 10*N)

    A= output1.windowed['No window'].A
    B= output1.windowed['No window'].B
    C= output1.windowed['No window'].C
    w0= output1.windowed['No window'].w0

    fitfunc1 = lambda t1: C + A * np.cos(w0*t1) + B * np.sin(w0*t1)

    axs3.plot(tt[:timePlot], output1.data[:timePlot],'.')
    axs3.plot(tt2[:timePlot*10], fitfunc1(tt2)[0:timePlot*10],'r-')
    axs3.grid(True)
    axs3.set_ylabel('Sample Value')
    axs3.set_xlabel('Frequency: %.4f MHz'%(inFreq/1000000))
    axs3.set_ylim([-128,128])

    A= output2.windowed['No window'].A
    B= output2.windowed['No window'].B
    C= output2.windowed['No window'].C
    w0= output1.windowed['No window'].w0

    fitfunc2 = lambda t2: C + A * np.cos(w0*t2) + B * np.sin(w0*t2)


    axs4.plot(tt[:timePlot], output2.data[:timePlot],'.')
    axs4.plot(tt2[:timePlot*10], fitfunc2(tt2)[0:timePlot*10],'r-')
    axs4.grid(True)
    axs4.set_ylabel('Sample Value')
    axs4.set_xlabel('Frequency: %.4f MHz'%(inFreq/1000000))
    axs4.set_ylim([-128,128])

    axs5.axis('tight')
    axs5.axis('off')

    colLabels = ('Parameter', 'Raw', 'w. Cal&GainOff')
    tableValues = [[elem[0],elem[1]%elem[2]] for elem in output1.items()]
    tableValues = tableValues+[[elem[0],elem[1]%elem[2]] for elem in output1.windowed['No window'].items()]
    tableValues2 = [[elem[1]%elem[2]] for elem in output2.items()]
    tableValues2 = tableValues2+[[elem[1]%elem[2]] for elem in output2.windowed['No window'].items()]
    tableHeaders = [elem[0] for elem in output2.items()]+[elem[0] for elem in output2.windowed['No window'].items()]
    tableVal = [elem[2] for elem in output2.items()]+[elem[2] for elem in output2.windowed['No window'].items()]
    tableValRaw = [elem[2] for elem in output1.items()]+[elem[2] for elem in output1.windowed['No window'].items()]
    for i in range(len(tableValues)):
        tableValues[i].append(tableValues2[i][0])
    printTableValues=[]
    for i in [0,1,2,3,5,6,7,8,9,11,13,14,16,17,20,21,22,23,24]:
        printTableValues.append(tableValues[i])
    the_table = axs5.table(cellText=printTableValues,colLabels =colLabels, loc='center')
    the_table.set_fontsize(8)
    the_table.scale(1, 1)
    if pdf==None and png:
        plt.savefig(fname+'.png')
    elif pdf != None:
        txt = 'fname: {}'.format(fname)
        plt.text(0.05,0.95,txt, transform=fig.transFigure, size=16)
        pdf.savefig(fig)
    if show:
        plt.show()
    plt.clf()
    #return tableHeaders, [[elem[2]] for elem in output2.items()]
    return tableHeaders, tableVal,tableValRaw


In [91]:
from IPython.display import display, clear_output

FOLDER = ('C:\\jupyter\\DATA\\SC\\SS_AMS\\')
FOLDER = ('C:\\jupyter\\DATA\\SC\\APPS\\')
FOLDER = ('C:\\jupyter\\DATA\\SC\\ADC_Char\\B009_08_2018c\\')
FOLDER = ('C:\\jupyter\\DATA\\SC\\ADC_Char\\B010_01_2018c\\PB6\\amp_sweep\\')
FOLDER = ('C:\\jupyter\\DATA\\SC\\ADC_Char\\B010_01_2018c\\PB6\\freq_sweep\\')
FOLDER = ('C:\\jupyter\\DATA\\SC\\ADC_Char\\B009_27_2018c\\PB6\\freq_sweep\\')
FOLDER_ROOT='Z:\\BRINGUP\\adc_measurements\\B0_Bringup\\PE\\ADC_Char_c\\'
FOLDER_ROOT='Z:\\BRINGUP\\adc_measurements\\B0_Bringup\\ADC_Char\\'

PARTS = glob.glob(FOLDER_ROOT+'*')
PARTS = [i.split('\\')[-1] for i in PARTS]

PARTS = ['S52_53G',
 'F51_53G',
 'F52_53G',
 'F53_53G',
 'F54_53G',
 'P6_53G',
 'PB10_53G',
 'PB13_53G',
 'PB29_53G',
 'PB30_53G',
 'PB5_53G',
 'PB6_53G',
 'PF53_53G',
 'PS51_53G',
 'PS53_53G',
 'S50_53G',
 'S51_53G',
 'S53_53G',
 'S54_53G',
 'T50_53G',
 'T51_53G',
 'T52_53G',
 'T53_53G',
 'T54_53G',
 'TT50_53G',
 'F50_53G']

PARTS = ['S52_51G',
 'F51_51G',
 'F52_51G',
 'F53_51G',
 'F54_51G',
 'S50_51G',
 'S51_51G',
 'S53_51G',
 'S54_51G',
 'T50_51G',
 'T51_51G',
 'T52_51G',
 'T53_51G',
 'T54_51G',
 'TT50_51G',
 'F50_51G']

PARTS = ['F53_816_85_53G']

FOLDER_ROOT='C:\\jupyter\\EVK\\logs\\233\\'
PARTS = ['skew1_coherent_0.2.5_m5dB']
SWEEPS = ['freq_sweep']

FOLDER_ROOT='C:\\jupyter\\EVK\\logs\\35_SC\\'
PARTS=['CRB_Fsweepcrb35_53G']
SWEEPS = ['freq_sweep']


#FOLDER = ('C:\\jupyter\\DATA\\SC\\ADC_Char\\B009_29_2018c\\PB6\\freq_sweep\\')

csvName=FOLDER_ROOT+'ENOB_ANALYSIS_CHAR_freq_sweep_CRB_Fsweepcrb35_53G.csv'

SWEEPS = ['freq_sweep']
#SWEEPS = ['amp_sweep']

headerWritten=False
header=[]

for PART in PARTS[::1]:
    PROCESS=PART[0:2]
    for SWEEP in SWEEPS:
        pdfName=FOLDER_ROOT+'{}_{}_ENOB_ANALYSIS_CHAR_freq_sweep_53G.pdf'.format(PART,SWEEP)
        with PdfPages(pdfName) as pdf:
            FOLDER = FOLDER_ROOT  + PART +  '\\' + SWEEP +'\\' 
            print(PART, FOLDER)
            fnames = glob.glob(FOLDER+'*.txt')

            if not headerWritten:
                try:
                    capture=np.loadtxt(fnames[0])
                    tableHeaders, tableVal, tableValRaw = procAll(capture,fname=fnames[0], samplingRate=10.3125*5*34/33*1.0e9,pdf=None,cal=True)
                    header=['fname','PROCESSING','PART_NO','PROCESS','SWEEP','UCID','BIN','ATTEN','AVDD','VDD08','TEMP','BOOST','CAL']+tableHeaders
                except:
                    pass
                fio=open(csvName,'w')
                for each in header:
                    fio.write(str(each)+',')
                fio.write('\n');fio.close()
                headerWritten=True

            for fname in fnames:
                clear_output(wait=True)
                capture=np.loadtxt(fname)
                if fname.find('Bin')>=0:
                    try:
                        UCID = int(fname.split('ucid_')[1].split('_')[0])
                        print('UCID: ',UCID, fname)
                        #PART_NO = 'PB'+fname.split('_PB')[1].split('_')[0]
                        PART_NO = PART
                        BIN = fname.split('Bin')[1].split('_')[0]
                        ATTEN = fname.split('dBm')[0].split('_')[-1]
                        ATTEN = ATTEN.replace('m-','-')
                        ATTEN = ATTEN.replace('m0','0')
                        AVDD = fname.split('avdd')[1].split('_')[0]
                        TEMP = fname.split('_ucid')[0].split('_')[-1]
                        TEMP = TEMP.replace('m','-')
                        BOOST = fname.split('boost_')[1].split('_')[0]
                        VDD08 = 'NOM'
                    except:
                        try:
                            UCID = 3
                            print('UCID: ',UCID, fname)
                            PART_NO = PART
                            BIN = fname.split('Bin')[1].split('_')[0]
                            ATTEN = 0
                            AVDD = fname.split('AVDD10_')[1].split('_')[0]
                            VDD08 = fname.split('VDD08_')[1].split('_')[0]
                            TEMP = 'ROOM'
                            BOOST = 0
                        except:
                            UCID = 3
                            print('UCID: ',UCID, fname)
                            PART_NO = PART
                            BIN = fname.split('Bin')[1].split('_')[0]
                            ATTEN = 0
                            AVDD = 'NOM'
                            VDD08 = 'NOM'
                            TEMP = 'ROOM'
                            BOOST = 0
                        
                    try:
                        if UCID == 3:
                            tableHeaders, tableVal, tableValRaw = procAll(capture,fname=fname, samplingRate=10.3125*5*34/33*1.0e9,pdf=pdf,cal=True)
                        if UCID == 1:
                            tableHeaders, tableVal, tableValRaw = procAll(capture,fname=fname, samplingRate=10.3125*5*1.0e9,pdf=pdf,cal=True)
                        data=[]
                        data.append([fname,'OK',PART_NO,PROCESS,SWEEP,UCID,BIN,ATTEN,AVDD,VDD08,TEMP,BOOST,0]+tableValRaw)
                        data.append([fname,'OK',PART_NO,PROCESS,SWEEP,UCID,BIN,ATTEN,AVDD,VDD08,TEMP,BOOST,1]+tableVal)
                        for DATA in data:
                            fio=open(csvName,'a+')
                            for each in DATA:
                                fio.write(str(each)+',')
                            fio.write('\n');fio.close()
                    except:
                        data=[]
                        data.append([fname,'ERROR',PART_NO,PROCESS,SWEEP,UCID,BIN,ATTEN,AVDD,VDD08,TEMP,BOOST,0])
                        data.append([fname,'ERROR',PART_NO,PROCESS,SWEEP,UCID,BIN,ATTEN,AVDD,VDD08,TEMP,BOOST,1])
                        for DATA in data:
                            fio=open(csvName,'a+')
                            for each in DATA:
                                fio.write(str(each)+',')
                            fio.write('\n');fio.close()


UCID:  3 C:\jupyter\EVK\logs\35_SC\CRB_Fsweepcrb35_53G\freq_sweep\Bin991_m0p0dBm_avdd1p0_crb35_53G_25_ucid_3_boost_0_freq.txt
991 990
0.0011699640899298987
0.0011685968664163291
990.9988314031336
Sinefit: 0
Sinefit: 1
Sinefit: 2
calculating DNL INL
calculating DNL INL 1.7679191468819107 34.91972951789485
991 990
0.0011699640899298987
0.0011685968664163291
990.9988314031336
Sinefit: 0
Sinefit: 1
Sinefit: 2
calculating DNL INL
calculating DNL INL 1.7679191468819107 34.91972951789485
Number of bits 8 b
Sampling rate 53125000000.00 Hz
Number of samples 16384 samples
Input frequency [0] 3213309838.73018 Hz
Peak [0] 990
Amplitude [0] 104.691899
Phase [0] -1.528383
DC [0] 1.254642
Max DNL 1.767919 
Max INL 34.919730 
Theoretical SNR 49.92 dB
Process Gain 6.02 dB
Input frequency [W] 3213309838.73018 Hz
Peak [W] 990
Amplitude [W] 104.691899
Phase [W] -1.528383
DC [W] 1.254642
THD -5.98 dB
Noise floor 30.0348 dB
Signal power 45033402 
Noise power 45007534 
SNR 0.00 dB
SINAD/SNDR 26.85 dB
ENOB 4.

<Figure size 1332x900 with 0 Axes>

In [68]:
def plotHistogram(fig, ax, DATA, binSize=1, ylim=500, title='', log=False, alpha=1, color='b'):
    DATA_hist=np.histogram(DATA, bins=range(-128, 128+1, binSize))
    x=np.arange(-128,128,binSize)
    ylim=ylim*binSize

    ax.bar(x,DATA_hist[0], color=color, alpha=alpha)
    ax.set_title(title)
    ax.set_xlabel('Bin')
    ax.set_ylabel('Number of Samples')
    ax.set_xticks(x[::8])
    props = {"rotation" : 90}
    plt.setp(ax.get_xticklabels(), **props)
    #plt.xticks(rotation=90)
    ax.set_ylim(0,ylim)
    if log:
        ax.set_ylim(1,ylim)
        ax.set_yscale('log')


In [22]:
np.mean([1,2,3])

2.0

In [92]:
FOLDER_ROOT='Z:\\BRINGUP\\adc_measurements\\B0_Bringup\\ADC_Char\\'

PARTS = glob.glob(FOLDER_ROOT+'*')
PARTS = [i.split('\\')[-1] for i in PARTS]

PARTS = ['S52_53G',
 'F51_53G',
 'F52_53G',
 'F53_53G',
 'F54_53G',
 'P6_53G',
 'PB10_53G',
 'PB13_53G',
 'PB29_53G',
 'PB30_53G',
 'PB5_53G',
 'PB6_53G',
 'PF53_53G',
 'PS51_53G',
 'PS53_53G',
 'S50_53G',
 'S51_53G',
 'S53_53G',
 'S54_53G',
 'T50_53G',
 'T51_53G',
 'T52_53G',
 'T53_53G',
 'T54_53G',
 'TT50_53G',
 'F50_53G']


PARTS = ['S52_51G',
 'F51_51G',
 'F52_51G',
 'F53_51G',
 'F54_51G',
 'S50_51G',
 'S51_51G',
 'S53_51G',
 'S54_51G',
 'T50_51G',
 'T51_51G',
 'T52_51G',
 'T53_51G',
 'T54_51G',
 'TT50_51G',
 'F50_51G']

PARTS = ['F53_816_85_53G']


FOLDER_ROOT='C:\\jupyter\\EVK\\logs\\233\\'
PARTS = ['skew1_coherent_0.2.5_m2dB']
SWEEPS = ['freq_sweep']

FOLDER_ROOT='C:\\jupyter\\EVK\\logs\\35_SC\\'
PARTS=['CRB_Fsweepcrb35_53G']
SWEEPS = ['freq_sweep']


BINSS = [[2153,2113], [3347,3391], [1693, 1741], [5039, 5119], [881,941], [251,269]]

for PART in PARTS[::1]:
    PROCESS=PART[0:2]
    pdfName=FOLDER_ROOT+'HIST_OVERLAP_ANALYSIS_CHAR_{}_ALL.pdf'.format(PART)
    with PdfPages(pdfName) as pdf:
        for SWEEP in SWEEPS:
            FOLDER = FOLDER_ROOT  + PART +  '\\' + SWEEP +'\\' 
            fnames = glob.glob(FOLDER+'Bin*.txt')
            BINS = np.sort(np.unique([fname.split('Bin')[1].split('_')[0] for fname in fnames]))
            BINS = np.sort([int(i) for i in BINS])
            BINSS = [[BINS[i],BINS[i+1]] for i in range(len(BINS)-1)]
            print(BINSS)
            for BINS in BINSS:
                print(BINS)
                FOLDER = FOLDER_ROOT  + PART +  '\\' + SWEEP +'\\' 
                print(BINS[0])
                fnames = glob.glob(FOLDER+'Bin{}_*.txt'.format(BINS[0]))
                for each in fnames:
                    print(each)
                
                for fname in fnames:
                    print(fname, fname.replace('Bin{}'.format(BINS[0]),'Bin{}'.format(BINS[1])))
                    fig=plt.figure(figsize=(32,16))

                    DATA = np.loadtxt(fname)
                    DATA0 = DATA

                    ax1=fig.add_subplot(2,1,1)
                    ax2=fig.add_subplot(2,1,2)

                    plotHistogram(fig, ax1, DATA, alpha=.5, binSize=1, ylim=500, title='')
                    plotHistogram(fig, ax2, DATA, alpha=.5, binSize=1, ylim=500, title='', log=True)

                    fname=fname.replace('Bin{}'.format(BINS[0]),'Bin{}'.format(BINS[1]))
                    DATA = np.loadtxt(fname)

                    plotHistogram(fig, ax1, DATA, alpha=.5, color='r', binSize=1, ylim=500, title='Part {} Bin {} Blue {} Red DC {} {}'.format(PART,BINS[0],BINS[1],np.mean(DATA0),np.mean(DATA)))
                    plotHistogram(fig, ax2, DATA, alpha=.5, color='r', binSize=1, ylim=500, title='Part {} Bin {} Blue {} Red DC {} {}'.format(PART,BINS[0],BINS[1],np.mean(DATA0),np.mean(DATA)), log=True)
                    #plt.show()
                    #raw_input()
                    txt = 'fname: {} {}'.format(fname, BINS)
                    plt.text(0.05,0.95,txt, transform=fig.transFigure, size=12)
                    pdf.savefig(fig)
                    plt.clf()
            

[[1, 11], [11, 13], [13, 17], [17, 19], [19, 23], [23, 29], [29, 31], [31, 37], [37, 41], [41, 53], [53, 59], [59, 67], [67, 79], [79, 83], [83, 97], [97, 101], [101, 103], [103, 107], [107, 109], [109, 113], [113, 127], [127, 167], [167, 227], [227, 251], [251, 269], [269, 313], [313, 347], [347, 383], [383, 419], [419, 457], [457, 491], [491, 521], [521, 563], [563, 607], [607, 647], [647, 691], [691, 739], [739, 773], [773, 839], [839, 881], [881, 941], [941, 991], [991, 1051], [1051, 1097], [1097, 1151], [1151, 1193], [1193, 1237], [1237, 1279], [1279, 1321], [1321, 1373], [1373, 1439], [1439, 1487], [1487, 1523], [1523, 1567], [1567, 1601], [1601, 1637], [1637, 1693], [1693, 1741], [1741, 1783], [1783, 1823], [1823, 1877], [1877, 1933], [1933, 1973], [1973, 2017], [2017, 2063], [2063, 2113], [2113, 2153], [2153, 2203], [2203, 2251], [2251, 2311], [2311, 2371], [2371, 2411], [2411, 2447], [2447, 2503], [2503, 2557], [2557, 2593], [2593, 2621], [2621, 2659], [2659, 2689], [2689, 272

c:\python36\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  


[11, 13]
11
C:\jupyter\EVK\logs\35_SC\CRB_Fsweepcrb35_53G\freq_sweep\Bin11_m0p0dBm_avdd1p0_crb35_53G_25_ucid_3_boost_0_freq.txt
C:\jupyter\EVK\logs\35_SC\CRB_Fsweepcrb35_53G\freq_sweep\Bin11_m0p0dBm_avdd1p0_crb35_53G_25_ucid_3_boost_0_freq.txt C:\jupyter\EVK\logs\35_SC\CRB_Fsweepcrb35_53G\freq_sweep\Bin13_m0p0dBm_avdd1p0_crb35_53G_25_ucid_3_boost_0_freq.txt
[13, 17]
13
C:\jupyter\EVK\logs\35_SC\CRB_Fsweepcrb35_53G\freq_sweep\Bin13_m0p0dBm_avdd1p0_crb35_53G_25_ucid_3_boost_0_freq.txt
C:\jupyter\EVK\logs\35_SC\CRB_Fsweepcrb35_53G\freq_sweep\Bin13_m0p0dBm_avdd1p0_crb35_53G_25_ucid_3_boost_0_freq.txt C:\jupyter\EVK\logs\35_SC\CRB_Fsweepcrb35_53G\freq_sweep\Bin17_m0p0dBm_avdd1p0_crb35_53G_25_ucid_3_boost_0_freq.txt
[17, 19]
17
C:\jupyter\EVK\logs\35_SC\CRB_Fsweepcrb35_53G\freq_sweep\Bin17_m0p0dBm_avdd1p0_crb35_53G_25_ucid_3_boost_0_freq.txt
C:\jupyter\EVK\logs\35_SC\CRB_Fsweepcrb35_53G\freq_sweep\Bin17_m0p0dBm_avdd1p0_crb35_53G_25_ucid_3_boost_0_freq.txt C:\jupyter\EVK\logs\35_SC\CRB_Fsweep

[251, 269]
251
C:\jupyter\EVK\logs\35_SC\CRB_Fsweepcrb35_53G\freq_sweep\Bin251_m0p0dBm_avdd1p0_crb35_53G_25_ucid_3_boost_0_freq.txt
C:\jupyter\EVK\logs\35_SC\CRB_Fsweepcrb35_53G\freq_sweep\Bin251_m0p0dBm_avdd1p0_crb35_53G_25_ucid_3_boost_0_freq.txt C:\jupyter\EVK\logs\35_SC\CRB_Fsweepcrb35_53G\freq_sweep\Bin269_m0p0dBm_avdd1p0_crb35_53G_25_ucid_3_boost_0_freq.txt
[269, 313]
269
C:\jupyter\EVK\logs\35_SC\CRB_Fsweepcrb35_53G\freq_sweep\Bin269_m0p0dBm_avdd1p0_crb35_53G_25_ucid_3_boost_0_freq.txt
C:\jupyter\EVK\logs\35_SC\CRB_Fsweepcrb35_53G\freq_sweep\Bin269_m0p0dBm_avdd1p0_crb35_53G_25_ucid_3_boost_0_freq.txt C:\jupyter\EVK\logs\35_SC\CRB_Fsweepcrb35_53G\freq_sweep\Bin313_m0p0dBm_avdd1p0_crb35_53G_25_ucid_3_boost_0_freq.txt
[313, 347]
313
C:\jupyter\EVK\logs\35_SC\CRB_Fsweepcrb35_53G\freq_sweep\Bin313_m0p0dBm_avdd1p0_crb35_53G_25_ucid_3_boost_0_freq.txt
C:\jupyter\EVK\logs\35_SC\CRB_Fsweepcrb35_53G\freq_sweep\Bin313_m0p0dBm_avdd1p0_crb35_53G_25_ucid_3_boost_0_freq.txt C:\jupyter\EVK\logs

[1237, 1279]
1237
C:\jupyter\EVK\logs\35_SC\CRB_Fsweepcrb35_53G\freq_sweep\Bin1237_m0p0dBm_avdd1p0_crb35_53G_25_ucid_3_boost_0_freq.txt
C:\jupyter\EVK\logs\35_SC\CRB_Fsweepcrb35_53G\freq_sweep\Bin1237_m0p0dBm_avdd1p0_crb35_53G_25_ucid_3_boost_0_freq.txt C:\jupyter\EVK\logs\35_SC\CRB_Fsweepcrb35_53G\freq_sweep\Bin1279_m0p0dBm_avdd1p0_crb35_53G_25_ucid_3_boost_0_freq.txt
[1279, 1321]
1279
C:\jupyter\EVK\logs\35_SC\CRB_Fsweepcrb35_53G\freq_sweep\Bin1279_m0p0dBm_avdd1p0_crb35_53G_25_ucid_3_boost_0_freq.txt
C:\jupyter\EVK\logs\35_SC\CRB_Fsweepcrb35_53G\freq_sweep\Bin1279_m0p0dBm_avdd1p0_crb35_53G_25_ucid_3_boost_0_freq.txt C:\jupyter\EVK\logs\35_SC\CRB_Fsweepcrb35_53G\freq_sweep\Bin1321_m0p0dBm_avdd1p0_crb35_53G_25_ucid_3_boost_0_freq.txt
[1321, 1373]
1321
C:\jupyter\EVK\logs\35_SC\CRB_Fsweepcrb35_53G\freq_sweep\Bin1321_m0p0dBm_avdd1p0_crb35_53G_25_ucid_3_boost_0_freq.txt
C:\jupyter\EVK\logs\35_SC\CRB_Fsweepcrb35_53G\freq_sweep\Bin1321_m0p0dBm_avdd1p0_crb35_53G_25_ucid_3_boost_0_freq.txt C:

...

<Figure size 2304x1152 with 0 Axes>

<Figure size 2304x1152 with 0 Axes>

<Figure size 2304x1152 with 0 Axes>

<Figure size 2304x1152 with 0 Axes>

<Figure size 2304x1152 with 0 Axes>

In [57]:
BINS

[1, 11]